# Interactive Notebook: Network Assessment for Active Mobility

This Jupyter-Notebook guides through several network assessments based on NetAScore output.
In order to execute this Notebook, please make sure to have all requirements fulfilled - please follow the instructions in [README.md](README.md).

If you do not have a NetAScore output file at hand, please download one of the example files from https://doi.org/10.5281/zenodo.10886962 and place it inside the subdirectory `NetAScore/data/`.

In [ ]:
# import required packages
import geopandas as gpd
import networkx as nx
import os

# settings
case_id = "at_salzburg"

# computed properties
net_file = os.path.join("NetAScore", "data", f"netascore_{case_id}.gpkg")

## Load the network and start exploring bikeability...
Here, we load the network which was processed by NetAScore and output the first 10 rows (edges).

In [ ]:
net = gpd.read_file(net_file, layer="edge")
net.head()

Let's filter the network to only show segments with high bikeability and display the result on a map.

In [ ]:
net[net.index_bike_ft > 0.75].explore()

We can also choose a different basemap with reduced color scheme:
(see online documentation for more options: [GeoDataFrame.explore()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.explore.html))

In [ ]:
net[net.index_bike_ft > 0.75].explore(tiles="CartoDB Positron")

...and we can further apply a color scheme to plot the high-bikeability according to their bikeability value:

In [ ]:
net[net.index_bike_ft > 0.75].explore(column="index_bike_ft", cmap="Blues", tiles="CartoDB Positron")

## Let's prepare our network for routing...
First, we need to generate a directed graph from our input data. Having an input edge (A<->B) with an index_ft and index_tf will result in two rows: (A->B, index_ft) and (B->A, index_tf). 
Furthermore, we want to not only compute shortest paths, but allow for computing bikeable routes as well. For this, we need to compute a routing cost per segment. This can be regarded as perceived distance: better suitability means perceived distance is close to real distance, low suitability results in perceived distance is (1+ROUTING_FACTOR) * length (with default setting: up to five times real segment length).
All of these steps are outsourced into a function `netascore_to_routable_net` which is defined in [algo/net_helper.py](algo/net_helper.py).

To execute this step, we simply call the function as follows:

In [ ]:
import algo.net_helper as nh
net_routing = nh.netascore_to_routable_net(net)

Next, we have to create a NetworkX graph object from our edge list representation of the network.
We use a `MultiDiGraph` to allow for multiple directed links between nodes:

In [ ]:
g = nx.from_pandas_edgelist(net_routing, source='from_node', target='to_node', 
                            edge_attr=True, create_using=nx.MultiDiGraph, edge_key="edge_id")
g

## Load and display the nodes layer
As basis for computing routes between pairs of nodes of the network, we will first load and display the nodes layer.

In [ ]:
nodes = gpd.read_file(net_file, engine='pyogrio', fid_as_index=True, layer="node")
# filter for largest connected component
n_con_comp = max(nx.weakly_connected_components(g), key=len)
print(f"Largest connected component: {len(n_con_comp)} nodes out of {len(g)}")
nodes = nodes.loc[list(n_con_comp)]
nodes.explore(tiles="CartoDB Positron", marker_kwds={"radius":2}, style_kwds={"stroke":False})

Next, we can manually select an origin node as well as a destination node from the interactive map and assign the respective nodeIDs to the variables `from_node` and `to_node`.

In [ ]:
from_node = 21181  #20741
to_node = 23302  #25235

## Compute routes
Now, we can compute routes...
### General routing approach

In [ ]:
npath = nx.shortest_path(g, from_node, to_node) # this call results in computing the shortest (lowest number of passed segments) path
npath

For visualization, we need to retrieve an edge sequence from the node sequence returned by `nx.shortest_path`:

In [ ]:
# get edge sequence (edge IDs)
def get_epath(nseq):
    ep = []
    for i in range(len(nseq)-1):
        dta = g.get_edge_data(nseq[i], nseq[i+1])
        eid = list(dta.keys())[0]
        ep.append(eid)
    return ep

epath = get_epath(npath)
epath

In [ ]:
m = net.loc[epath].explore(tiles="CartoDB Positron")
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

### Shortest distance path
In order to retrieve the shortest distance path, we need to specify the edge weight column - in this case we use the "length" attribute.

In [ ]:
shortest_dist_path = get_epath(nx.shortest_path(g, from_node, to_node, weight="length"))
m = net_routing.loc[shortest_dist_path].explore(tiles="CartoDB Positron", tooltip=["cost_bike_ft"])
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

### Bikeable path
Here, we use the `cost_bike_ft` column which contains our pre-computed "perceived distance" value.

In [ ]:
bikeable_path = get_epath(nx.shortest_path(g, from_node, to_node, weight="cost_bike_ft"))
m = net.loc[bikeable_path].explore(tiles="CartoDB Positron")
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

## Additionally retrieve edge length and bikeability values
To allow retrieving additional attributes, we need to keep track of the direction of edge traversal in addition to the edge ID.

In [ ]:
import pandas as pd
# get edge sequence including "inverted" attribute (direction of traversal)
def get_epath_dir(nseq):
    ep = []
    inv = []
    for i in range(len(nseq)-1):
        dta = g.get_edge_data(nseq[i], nseq[i+1])
        eid = list(dta.keys())[0]
        ep.append(eid)
        inv.append(dta[eid]["inv"])
        #print(eid,dta[eid]["inv"])
    return pd.DataFrame(data={"edge_id":ep, "inv":inv})


In [ ]:
bp = get_epath_dir(nx.shortest_path(g, from_node, to_node, weight="cost_bike_ft"))
bp.head(5)

Now we have to prepare the "net_routing" DataFrame for allowing join operations with the routing result. For this, we now need to have a MultiIndex, allowing to jointly query/filter based on `edge_id` and `inv`.

In [ ]:
net_routing.head(5)

In [ ]:
net_join = net_routing.set_index(["edge_id", "inv"])
net_join.head(5)

In the following step we can join the path result with the just prepared network data by the joint key, using the edge ID and direction (inv).

In [ ]:
bp_joined = net_join.join(bp.set_index(["edge_id", "inv"]), how="right")
bp_joined.head(5)

This now allows us to plot additional route characteristics - here, the bikeability of each segment:

In [ ]:
m = bp_joined.explore(tiles="CartoDB Positron", column="index_bike_ft", cmap="RdYlGn", vmin=0, vmax=1)
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

...we may also filter the results based on bikeability - here, only very suitable segments (bikeability > 0.7) are shown:

In [ ]:
m = bp_joined[bp_joined.index_bike_ft > 0.7].explore(tiles="CartoDB Positron", column="index_bike_ft", cmap="RdYlGn", vmin=0, vmax=1)
nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2})

...and we can compute additional path statistics:

In [ ]:
def get_path_stats(path_df):
    length = path_df["length"].sum()
    length_weighted_avg_index = (path_df.index_bike_ft * path_df.length).sum() / path_df.length.sum()
    min_index = path_df.index_bike_ft.min()
    max_index = path_df.index_bike_ft.max()
    return pd.Series({"total_length": length, "length_weighted_avg_index": length_weighted_avg_index, 
         "min_index": min_index, "max_index": max_index})

In [ ]:
get_path_stats(bp_joined)

## Make the routing part easily re-usable

In [ ]:
def routing(from_node:int, to_node:int, compute_bikeable_path:bool, display_route:bool):
    weight_col = "cost_bike_ft"
    if not compute_bikeable_path:
        weight_col = "length"
    path = get_epath_dir(nx.shortest_path(g, from_node, to_node, weight=weight_col))
    net_join = net_routing.set_index(["edge_id", "inv"])
    path_joined = net_join.join(path.set_index(["edge_id", "inv"]), how="right")
    if display_route:
        m = path_joined.explore(tiles="CartoDB Positron", column="index_bike_ft", cmap="RdYlGn", vmin=0, vmax=1)
        display(nodes.loc[[from_node, to_node]].explore(m=m, color=["green", "red"], marker_kwds={"radius":5}, style_kwds={"weight":2}))
    return get_path_stats(path_joined)

In [ ]:
# compute shortest path
routing(from_node, to_node, False, True)

In [ ]:
#compute bikeable path
routing(from_node, to_node, True, True)

## Accessibility (single POI node)
In this example, we will compute several routes from a given POI node.
First, the origin node is defined. Then, a distance cutoff value is set and all nodes within this maximum radius are fetched. 

In [ ]:
orig_node = from_node
orig_node_geom = nodes.loc[orig_node].geometry
dist_cutoff = 2000 # in map units (meter)
#destination nodes: all nodes within distance limits
nodes_dist = nodes.geometry.distance(nodes.loc[orig_node].geometry)
node_ids_filtered = nodes_dist[nodes_dist <= dist_cutoff].index
dest_nodes = nodes.loc[node_ids_filtered]
dest_nodes["dist"] = nodes_dist
m = dest_nodes.explore(tiles="CartoDB Positron")
gpd.GeoDataFrame(pd.DataFrame(nodes.loc[orig_node]).T, crs=nodes.crs).explore(m=m, color=["red"],
                                                                              marker_kwds={"radius":5}, style_kwds={"weight":2})

We now draw a random sample from these nodes. These will be used for routing.

In [ ]:
node_sample_n = 500
if len(dest_nodes) > node_sample_n:
    dest_nodes = dest_nodes.sample(node_sample_n)
m = dest_nodes.explore(tiles="CartoDB Positron")
gpd.GeoDataFrame(pd.DataFrame(nodes.loc[orig_node]).T, crs=nodes.crs).explore(m=m, color=["red"],
                                                                              marker_kwds={"radius":5}, style_kwds={"weight":2})

Here, we define the routing function that will be called per destination node.

In [ ]:
def comp_single_route(node_id):
    return routing(orig_node, node_id, True, False)
# store node id in separate column
dest_nodes["node_id"] = dest_nodes.index

In the following step, the routing function is called per destination node and the results (route statistics) are joined to the node DataFrame.

In [ ]:
results = dest_nodes.join(dest_nodes.node_id.apply(comp_single_route))
results

Now we can visualize the results and assess various aspects - e.g. the length-weighted average bikeability, minimum and maximum bikeability per route

### Length-weighted average bikeability

In [ ]:
m = results.explore(column="length_weighted_avg_index", cmap="RdYlGn", vmin=0, vmax=1, tiles="CartoDB Positron")
gpd.GeoDataFrame(pd.DataFrame(nodes.loc[orig_node]).T, crs=nodes.crs).explore(m=m, color=["red"],
                                                                              marker_kwds={"radius":5}, style_kwds={"weight":2})

### Lowest bikeability per route

In [ ]:
m = results.explore(column="min_index", cmap="RdYlGn", vmin=0, vmax=1, tiles="CartoDB Positron")
gpd.GeoDataFrame(pd.DataFrame(nodes.loc[orig_node]).T, crs=nodes.crs).explore(m=m, color=["red"],
                                                                              marker_kwds={"radius":5}, style_kwds={"weight":2})

In [ ]:
m = results[results.min_index > 0.5].explore(tiles="CartoDB DarkMatter", column="length_weighted_avg_index", cmap="RdYlGn", vmin=0, vmax=1)
gpd.GeoDataFrame(pd.DataFrame(nodes.loc[orig_node]).T, crs=nodes.crs).explore(m=m, color=["red"],
                                                                              marker_kwds={"radius":5}, style_kwds={"weight":2})

### Highest bikeability per route

In [ ]:
m = results.explore(column="max_index", cmap="RdYlGn", vmin=0, vmax=1, tiles="CartoDB Positron")
gpd.GeoDataFrame(pd.DataFrame(nodes.loc[orig_node]).T, crs=nodes.crs).explore(m=m, color=["red"],
                                                                              marker_kwds={"radius":5}, style_kwds={"weight":2})

Furthermore, we can set a network distance limit (max. route length):

In [ ]:
# add buffer with r=dist_cutoff (straight-line distance)
m = gpd.GeoDataFrame({"idx":["buffer"],"geometry":[nodes.loc[orig_node].geometry.buffer(dist_cutoff)]}, crs=nodes.crs).explore(
        color=["#DEDEDE"],marker_kwds={"radius":5}, style_kwds={"weight":2}, tiles="CartoDB DarkMatter", popup=None, tooltip=None, interactive=False)
# add nodes within max. distance of dist_cutoff
results[results.total_length < dist_cutoff].explore(m=m, column="length_weighted_avg_index", cmap="RdYlGn", vmin=0, vmax=1)
# display origin node
gpd.GeoDataFrame(pd.DataFrame(nodes.loc[orig_node]).T, crs=nodes.crs).explore(m=m, color=["red"],
                                                                              marker_kwds={"radius":5}, style_kwds={"weight":2})

## Additional network characteristics

### Centrality metrics
As some of these measures have higher computational complexity, we use a subset of the network as input.
However, please be aware that subsetting and the choice of the area of interest have substantial impact on centrality results (except degree centrality).

In [ ]:
subset_nodes = nodes.loc[node_ids_filtered]
sub_g = g.subgraph(subset_nodes.index)


#### Nodes: degree centrality
This measure represents the number of links connected to each node

In [ ]:
c_deg = pd.DataFrame(sub_g.degree(), columns=["idx", "degree"]).set_index("idx")
subset_nodes.join(c_deg).explore(column="degree", tiles="CartoDB DarkMatter")

#### Nodes: closeness centrality
Closeness represents the average shortest path distance of all reachable nodes to each node.
Caution: this requires computing all shortest paths within the given network and therefore may result in long computation time.

In [ ]:
c_clo = pd.DataFrame.from_dict({"closeness": nx.closeness_centrality(sub_g, distance="length")})
subset_nodes.join(c_clo).explore(column="closeness", tiles="CartoDB DarkMatter")

### Edges: betweenness centrality
This is an example for computing standard edge betweenness centrality. It represents the share of shortest paths that pass each edge. 
The function `add_centr_to_netascore` is defined in [algo/net_helper.py](algo/net_helper.py) and abstracts the steps for direction-aware assignment of results per input edge.

In [ ]:
bc = nx.edge_betweenness_centrality(sub_g, k=250, weight="length")

In [ ]:
nh.add_centr_to_netascore(net, bc, "bc_sp")
# display centrality for segments with centr. > 1% of maximum centrality
net[net.centr_bc_sp_sum > net.centr_bc_sp_sum.max()*0.01].explore(column="centr_bc_sp_sum", 
                    tiles="CartoDB DarkMatter", interactive=False, popup=None, tooltip=None)

#### ...same for bikeable paths

In [ ]:
bc_bp = nx.edge_betweenness_centrality(sub_g, k=250, weight="cost_bike_ft")

In [ ]:
nh.add_centr_to_netascore(net, bc_bp, "bc_bp")
# display centrality for segments with centr. > 1% of maximum centrality
net[net.centr_bc_bp_sum > net.centr_bc_bp_sum.max()*0.01].explore(column="centr_bc_bp_sum", 
                    tiles="CartoDB DarkMatter", interactive=False, popup=None, tooltip=None)